<a href="https://colab.research.google.com/github/igor-stefan/metodos_numericos_2019/blob/master/interpolacao_implementacao.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

#Newton

In [ ]:
x=np.array([3.0,5.0,9.0],dtype=float)
y=np.array([40.0,82.0,214.0],dtype=float)


def newton(x,val_x,val_y):
  coeficientes = np.zeros(val_x.shape[0])
  m_val=np.zeros((val_x.shape[0],val_x.shape[0]),dtype=float)
  m_controle=np.zeros((val_x.shape[0],val_x.shape[0]),dtype=bool)
  res=0;
  diferenca=1;
  for i in range (coeficientes.shape[0]):
    coeficientes[i]=pd(0,i,m_val,m_controle,val_x,val_y)
    res+=diferenca*coeficientes[i]
    diferenca*=x-val_x[i]
  print('Valores dos coeficientes = ',coeficientes);
  print('f(',x,') = ', res,sep='')
  return res;

def pd(inicio,fim,m_val,m_controle,val_x,val_y):
  if m_controle[inicio,fim]:
    return m_val[inicio,fim] #se ja tiver sido calculado, retorna o valor referente
  if(inicio==fim):
    return val_y[inicio] #quando chegar num caso base, retornar o valor de f(x)
  #caso nao seja um caso base, fazer a pd de acordo com a formula de calculo das diferenças divididas
  #f[xi,xi+1,xi+2,...,xi+k-1,xi+k]= (f[xi,xi+1,xi+2,...,xi+k-1,xi+k]-f[xi,xi+1,xi+2,...xi+k-2,,xi+k-1])/(xi+k)-xi
  #o nome pd é devido à tecnica utilizada
  m_val[inicio,fim]=(pd(inicio+1,fim,m_val,m_controle,val_x,val_y)-pd(inicio,fim-1,m_val,m_controle,val_x,val_y))/(val_x[fim]-val_x[inicio])
  m_controle[inicio,fim]=1
  return m_val[inicio,fim];

newton(4.0,x,y)

# como eu posso escrever o polinomio interpolador como pn(x)= f[x0] + a1(x − x0)+ a2(x − x0)(x − x1) + ... + an(x − x0)(x − x1)...(x − xn−1)
# o valor de f[xo] se refere a primeira diferença dividida que é igual a f(x0). a1,a2,an... são as n-ésimas diferenças divididas que correspondem aos coeficientes
# por exemplo: a1 = f(x1)-f(x0)/x1-x0
# dessa maneira, em cada indice do loop, é calculado a diferença dividida referente a val_x[i] e ao resultado é adicionado o produto de an pelo valor da diferenca x-x[i]


Valores dos coeficientes =  [40. 21.  2.]
f(4.0) = 59.0


59.0

#Lagrange

In [ ]:
x=np.array([3.0,5.0,9.0],dtype=float)
y=np.array([40.0,82.0,214.0],dtype=float)

def lagrange(x,val_x,val_y):
  for i in range(val_x.shape[0]):
    if val_x[i]==x:
      return val_y[i]
  
  g=np.zeros((val_x.shape[0],val_x.shape[0])) #matriz G para metodo de lagrange utilizando dispositivo pratico
  gd=1 #produto da diagonal principal
  l=val_y.copy();

  #for para construir a matriz; onde i=j o valor é x-val_x[coluna]; onde i!=j, val_x[linha]-val_x[coluna]
  for i in range(g.shape[0]): 
    for j in range(g.shape[0]): 
      if i!=j:
        g[i,j]=val_x[i]-val_x[j]
      else:
        g[i,j]=x-val_x[j]
        gd*=g[i,j] #atualiza o produto da diagonal principal
  res=0;
  for i in range(g.shape[0]):
    l[i]=gd/np.prod(g[i,:]) #Cada Li(x) recebe o quociente entre o produto da diagonal principal e o produto da respectiva linha i.
    res+=l[i]*val_y[i] #o resultado é a soma de cada Li(x) com o respectivo val_y[x](ou valor de f(xi))

  print('f(',x,') = ',res)
  print('Coeficientes = ', l)
  return res

lagrange(6.0,x,y)


f( 6.0 ) =  109.0
Coeficientes =  [-0.25   1.125  0.125]


109.0

#Direto


In [ ]:
def pivotamento(matriz):
  for i in range(matriz.shape[0]):
    maior_abs=np.abs(matriz[i,i])
    maior=matriz[i,i]
    for q in range(i+1,matriz.shape[0]): 
      if(np.abs(matriz[q,i])>maior_abs): 
        maior=matriz[q,i] 
        matriz[[i,q]]=matriz[[q,i]] 
    pivo=maior 
    equacao_pivo=matriz[i,:] 
    j=i+1 
    while(j<matriz.shape[0]): 
      eliminador=matriz[j,i]/pivo 
      matriz[j,:]=matriz[j,:]-(eliminador*equacao_pivo) 
      j=j+1 
  return matriz 

def substituicao(matriz):
  solucoes=np.zeros(matriz.shape[0],dtype=float) 
  solucoes[-1]=matriz[-1,-1]/matriz[-1,-2] 
  soma=0
  for i in range(-(matriz.shape[0]-2),1): 
    for j in range(matriz.shape[0]): 
      if(j!=-i): 
        soma+=solucoes[j]*matriz[-i,j] 
    solucoes[-i]=(matriz[-i,-1]-soma)/matriz[-i,-i] 
    soma=0 
  return solucoes 

def gauss_pivotamento(matriz_sistema):
  return substituicao(pivotamento(matriz_sistema))

def direto(x,val_x,val_y):
  m=np.zeros((val_x.shape[0],val_x.shape[0]+1)) #preenche-se uma matriz de zeros
  m[:,-1]=val_y.copy()#ultima coluna recebe os valores de y
  for i in range(val_x.shape[0]):
    for j in range(val_x.shape[0]):
      m[i,j]=val_x[i]**j #x^n+x^n-1+x^n-2+...+x^2+x+0 = y
  coeficientes=gauss_pivotamento(m) #resolvo o sistema
  res=0
  for k in range(len(coeficientes)):
    res+=coeficientes[k]*x**k
  print('Coeficientes = ', coeficientes)
  print('f(',x,') = ', res,sep='')
  return res


x1=np.array([3.0,5.0,9.0],dtype=float)
y1=np.array([40.0,82.0,214.0],dtype=float)
h=4.0

direto(h,x1,y1)

Coeficientes =  [7. 5. 2.]
f(4.0) = 59.0


59.0

#Splines

In [ ]:
x=np.array([3.0,4.5,7.0,9.0],dtype=float) #valores de x da tabela
y=np.array([2.5,1.0,2.5,.5],dtype=float) #valores de y da tabela

#nome da função em referência ao amigo Rafael que ajudou-me e tambem outros colegas a entender o padrão de preenchimento da matriz no metodo splines.
def rafael_matriz_splines(val_x,val_y):
  matriz=np.zeros((3*(val_x.shape[0]-1),3*(val_x.shape[0]-1)+1)) #numero de linhas e colunas da matriz
  num_pol=val_x.shape[0]-1 #numero de polinomios para n pontos é n-1.

  for i in range(matriz.shape[0]):
    if i<2*num_pol: #até a linha equivalente a 2 vezes o numero de polinomios, a ultima coluna recebe os valores de f(x)
      ind=i//2 #padrão para colocar os indices do vetor dos dos valores de x.
      if i%2:
        #se a linha for impar
        matriz[i,ind]=val_x[ind+1]**2
        matriz[i,ind+num_pol]=val_x[ind+1]
        matriz[i,ind+2*num_pol]=1
      else:
        #se for par
        matriz[i,ind]=val_x[ind]**2
        matriz[i,ind+num_pol]=val_x[ind]
        matriz[i,ind+2*num_pol]=1
      matriz[i,-1]=val_y[(1+i)//2] #colocando os valores de f(x)
    elif i<(matriz.shape[0]-1):
      #preenchendo as condições das derivadas
      matriz[i,i-2*num_pol]=2*val_x[i+1-2*num_pol]
      matriz[i,i+1-2*num_pol]=-2*val_x[i+1-2*num_pol]
      matriz[i,i-num_pol]=1
      matriz[i,i+1-num_pol]=-1
    else:
      #a1=0
      matriz[i,0]=1
  return matriz;


#ms = matriz spline preenchida
def solucao_sistema(ms):
  a=ms[:,:-1].copy(); #separa a matriz dos coeficientes
  b=ms[:,-1].copy(); #matriz dos valores
  return np.linalg.solve(a,b)


#ssms = solução sistema matriz splines
#val_x = valores de x da tabela
#x = ponto intermediario que deseja encontrar f(x)

def polinomio_splines(x,val_x,ssms):
  k=val_x.shape[0]-1 #k é o numero de polinomios splines
  for j in range (k):
    if x<=val_x[j+1]: #se x está dentro do intervalo
      return ssms[j]*x**2+ssms[j+k]*x+ssms[j+k*2] #retorna o valor splines de x, da forma ax^2+bx+c

def splines(x,val_x,val_y):
  k=rafael_matriz_splines(val_x,val_y)
  k1=solucao_sistema(k)
  k2=polinomio_splines(x,val_x,k1)
  #print('f(',x,') = ',k2,sep='')
  return k2;

splines(4.1,x,y)

1.4000000000000004